In [5]:

from __future__ import print_function

import numpy as np
import tensorflow as tf
from tensorflow.contrib.factorization import KMeans

# Ignore all GPUs, tf random forest does not benefit from it.忽略所有的GPU，TF随机森林不从中受益。
# import os
# os.environ["CUDA_VISIBLE_DEVICES"] = ""

In [30]:

# Import MNIST data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("data/", one_hot=True)

# 所有图片组成的矩阵
full_data_x = mnist.train.images

#显示图片
# sum(full_data_x[0])

Extracting data/train-images-idx3-ubyte.gz
Extracting data/train-labels-idx1-ubyte.gz
Extracting data/t10k-images-idx3-ubyte.gz
Extracting data/t10k-labels-idx1-ubyte.gz


In [27]:
# Parameters  参数
num_steps = 50 # Total steps to train  培训的全部步骤
batch_size = 1024 # The number of samples per batch  每批样品的数量
k = 3 # The number of clusters  聚类数
num_classes = 10 # The 10 digits  10位数
num_features = 784 # Each image is 28x28 pixels

# Input images   加载图片
#Tensor("Placeholder_6:0", shape=(?, 784), dtype=float32)
'''
tf.placeholder(dtype, shape=None, name=None)

此函数可以理解为形参，用于定义过程，在执行的时候再赋具体的值
参数：

    dtype：数据类型。常用的是tf.float32,tf.float64等数值类型
    shape：数据形状。默认是None，就是一维值，也可以是多维，比如[2,3], [None, 3]表示列是3，行不定
    name：名称。
'''
X = tf.placeholder(tf.float32, shape=[None, num_features])

#Tensor("Placeholder_6:0", shape=(?, 784), dtype=float32)
# Labels (for assigning a label to a centroid and testing)  标签，分配一个标签给质心和测试
Y = tf.placeholder(tf.float32, shape=[None, num_classes])

# K-Means Parameters  K均值参数
'''
Idx,C,sumD,D]=Kmeans(data,3,’dist’,’sqEuclidean’,’rep’,4)


等号右边：

kmeans：K-均值聚类

data是你自己的输入数据

3 是你要聚成3类

dist sqEuclidean 这2个参数，表示距离函数为欧式距离。什么是欧式距离自己百度

’rep’,4 聚类重复次数4次。因为要反复算直到选出最好的结果，至多反复算4次


等号左边：

Idx 是你聚类的标号

C 是聚类之后质心的位置

sumD是所有点到质心的距离之和

D是每个点与所有质心的距离
'''
kmeans = KMeans(inputs=X, num_clusters=k, distance_metric='cosine',
                use_mini_batch=True)
print(kmeans)

In [39]:
# Build KMeans graph   建立目标图
(all_scores, cluster_idx, scores, cluster_centers_initialized, init_op,
train_op) = kmeans.training_graph()
# print(train_op)


#(<tf.Tensor 'Squeeze_25:0' shape=<unknown> dtype=int64>,)
# print(cluster_idx)


#(<tf.Tensor 'sub_29:0' shape=(?, ?) dtype=float32>,)
# print(all_scores)

#(<tf.Tensor 'Squeeze_16:0' shape=<unknown> dtype=float32>,)
# print(scores)

#<tf.Variable 'initialized_9:0' shape=() dtype=bool_ref>
# print(cluster_centers_initialized)


#Tensor("cond_10/Merge:0", shape=(), dtype=bool)
# print(init_op)


# print(train_op)

# 转化为一个张量Tensor("Squeeze_25:0", dtype=int64)
cluster_idx = cluster_idx[0] # fix for cluster_idx being a tuple  修正了cluster_idx作为一个元组
# print(cluster_idx)


#求每个聚类的平均值 新的中心点
avg_distance = tf.reduce_mean(scores)

#Tensor("Mean_20:0", dtype=float32)
# print(avg_distance)

In [40]:
# Start TensorFlow session
with tf.Session() as sess:
# Run the initializer（运行初始化）

    tf.global_variables_initializer().run()
    
    sess.run(init_op, feed_dict={X: full_data_x})
    
    #Tensor("cond_13/Merge:0", shape=(), dtype=bool)
#     print(init_op)

# Training（训练）
    for i in range(1, num_steps + 1):
#     for i in range(num_steps):

        _, d, idx = sess.run([train_op, avg_distance, cluster_idx],
                             feed_dict={X: full_data_x})
        if i % 10 == 0 or i == 1:
            print("Step %i, Avg distance: %f" % (i, d))
    
    
    #[0 2 1 ..., 2 2 1]
#     print(idx)
    
    #0.155479
#     print(d)
    
    
    # Assign a label to each centroid(给每一个标签分配一个质心)
    # Count total number of labels per centroid, using the label of each training（计算每个质心的标签总数，使用每个培训的标签。）
    # sample to their closest centroid (given by 'idx')（样本最近的质心（由“编号”））
    
    
    '''
    [[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
     [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
     [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]]
    '''
    counts = np.zeros(shape=(k, num_classes))
#     print(counts)
    
    for i in range(len(idx)):
        counts[idx[i]] += mnist.train.labels[i]
        
        
    # [2, 7, 1]
    # Assign the most frequent label to the centroid(给质心分配最频繁的标签)
    labels_map = [np.argmax(c) for c in counts]
#     print(labels_map)
    
    # Tensor("Const_17:0", shape=(3,), dtype=int32)
    labels_map = tf.convert_to_tensor(labels_map)
#     print(labels_map)

    # Evaluation ops
    # Lookup: centroid_id -> label
    '''
     tf.nn.embedding_lookup
     第一个参数为所要转变为列向量的行向量，第二个参数为排列的行向量的索引号
    '''
    
    # Tensor("embedding_lookup_7:0", dtype=int32) 每行那就标签列向量
    cluster_label = tf.nn.embedding_lookup(labels_map, cluster_idx)
#     print(cluster_label)
    '''
    求平均值tf.reduce_mean(input_tensor, reduction_indices=None, keep_dims=False, name=None)

    参数1--input_tensor:待求值的tensor。

    参数2--reduction_indices:在哪一维上求解。

    参数（3）（4）可忽略
    求均值，tf.reduce_mean(x,0）列求均值  tf.reduce_mean(x,1)行求均值


    cast(x, dtype, name=None)
    将x的数据格式转化成dtype.例如，原来x的数据格式是bool，
    那么将其转化成float以后，就能够将其转化成0和1的序列。反之也可以
    '''
    
    # Tensor("Equal_12:0", dtype=bool)
    correct_prediction = tf.equal(cluster_label, tf.cast(tf.argmax(Y, 1), tf.int32))
#     print(correct_prediction)
    
    
    # Tensor("Mean_12:0", dtype=float32)
    accuracy_op = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
#     print(accuracy_op)

    # Test Model  加载测试集的图片标签
    test_x, test_y = mnist.test.images, mnist.test.labels
    
    print("测试精度:", sess.run(accuracy_op, feed_dict={X: test_x, Y: test_y}))

Tensor("cond_13/Merge:0", shape=(), dtype=bool)
Step 1, Avg distance: 0.253534
Step 10, Avg distance: 0.156138
Step 20, Avg distance: 0.155795
Step 30, Avg distance: 0.155643
Step 40, Avg distance: 0.155548
Step 50, Avg distance: 0.155479
测试精度: 0.2942
